# Basic Tools for Alhazen

> Simple tools to demonstrate utility and 'agentic' functionality. 

## Available Tools:

### Building, Describing, and Deleting Collections:
1. **AddCollectionFromEPMCTool**: Execute a query against the European PMC, creating a collection, downloading available full-text papers and saving expressions, items, and fragments to the database to denote that publication.
2. **DescribeCollectionCompositionTool**: Describe the composition of a collection in the database.
3. **DeleteCollectionTool**: Delete a collection from the database.

### Searching for papers in collections:
3. **List__Paged_Expressions_CollectionCTool**: Delete a collection from the database.

## Tools under development:
1. Develop queries across external data sources
2. Extract information from a collection using an LLM-based analysis to create Notes 
3. Filter a collection by an LLM-based analysis by tagging fragments with Notes
5. Report on the state of the database in terms of numbers of collections, expressions, items, and fragments
6. Prepare a report over a core research question by collecting a number of notes and synthesizing them into a report

In [ ]:
#| default_exp tools.basic

In [1]:
#| export

import os

from alhazen.core import PromptTemplateRegistry, load_alhazen_tool_environment
from alhazen.utils.searchEngineUtils import download_full_text_paper_for_doi
from alhazen.utils.web_robot import *
from alhazen.schema_sqla import *
from alhazen.utils.ceifns_db import Ceifns_LiteratureDb

import json

from langchain.tools import StructuredTool
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

from langchain.agents import AgentType, initialize_agent
from langchain.agents.agent import RunnableAgent
from langchain.chains import LLMMathChain
from langchain.chat_models.base import BaseChatModel
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain.llms.base import BaseLanguageModel
from langchain.pydantic_v1 import BaseModel, Field
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import JsonOutputParser

from operator import itemgetter
import pandas as pd

from sqlalchemy import create_engine, exists, func, or_
from sqlalchemy.orm import sessionmaker, aliased

from typing import Optional, Type

In [ ]:
#| export

skc = aliased(ScientificKnowledgeCollection)
skc_hm = aliased(ScientificKnowledgeCollectionHasMembers)
ske = aliased(ScientificKnowledgeExpression)
ske_hr = aliased(ScientificKnowledgeExpressionHasRepresentation)
ski = aliased(ScientificKnowledgeItem)
ski_hp = aliased(ScientificKnowledgeItemHasPart)
skf = aliased(ScientificKnowledgeFragment)

In [2]:
#| export
# NOTE - Use LangChain's SQL_DATABASE TOOL AS A MODEL 
# https://github.com/langchain-ai/langchain/blob/535db72607c4ae308566ede4af65295967bb33a8/libs/community/langchain_community/tools/sql_database/tool.py

class AlhazenToolMixin(BaseModel):
    '''Base tool for interacting with an Alhazen CEIFNS (pron. 'SAI-FiNS') database 
    (CEIFNS = Collection-Expression-Item-Fragment-Note-Summary).'''

    db: Ceifns_LiteratureDb = Field(exclude=True)
    llm: Optional[BaseChatModel] = Field(exclude=True)

    class Config:
        """Configuration for this pydantic object."""
        arbitrary_types_allowed = True

class IntrospectionToolMixin(BaseModel):
    '''Base class for providing information about Alhazen's capabilities.'''

    agent: Optional[RunnableAgent] = Field(exclude=True)

    class Config:
        """Configuration for this pydantic object."""
        arbitrary_types_allowed = True

In [ ]:
#| export

class IntrospectionToolSchema(BaseModel):
    input: str = Field(description="A hypothetical input to an agent.")

class IntrospectionTool(IntrospectionToolMixin, BaseTool): 
    name = "introspect"
    description = "This tool permits Alhazen to answer questions how its agent would plan to answer specific questions."
    args_schema: Type[IntrospectionToolSchema] = IntrospectionToolSchema

    def _set_agent(self, agent):
        self.agent = agent

    def _run(
        self,
        input: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:

        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # 1. Check to see if the agent was instantiated
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        if self.agent is None:
            raise Exception('Introspection tool not instantiated with agent')
        
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # 2. Run agent's plan method 
        #~~~~~~~~~~~~~~~~~~~~~~~~~~
        payload = {'intermediate_steps': [], 'input': input}
        output = self.agent.invoke(payload)

        return output

    async def _arun(
        self,
        name: str,
        query: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("add_collection_from_epmc_query does not support async")

In [3]:
#| export

class AddCollectionFromEPMCToolSchema(BaseModel):
    id: str = Field(description="A code that serves as the id of the collection we will add papers to.")
    name: str = Field(description="A human-readable name of the collection we will add papers to.")
    query: str = Field(description="A search query (using AND/OR/NOT statements) to be executed on the European PMC online literature repository for scientific publications.")

class AddCollectionFromEPMCTool(AlhazenToolMixin, BaseTool): 
    name = "add_collection_from_epmc_query"
    description = "This tool constructs a collection in the database from an external source based on a search query ."
    args_schema: Type[AddCollectionFromEPMCToolSchema] = AddCollectionFromEPMCToolSchema
    return_direct:bool = True

    def _run(
        self,
        id: str, 
        name: str,
        query: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        
        n_papers_added = 0

        try: 
            
            c = self.db.add_corpus_from_epmc_query(id, name, query, commit_this=False)
            
            # Need to add provenance for the data we have added to the model. 
            c.provenance = json.dumps({'action_type': self.name, 'action': {'id': id, 'name': name, 'query': query} })
            self.db.session.flush()

            #q2 = self.db.session.query(ske) \
            #        .filter(skc.id == skc_hm.ScientificKnowledgeCollection_id) \
            #        .filter(skc_hm.has_members_id == ske.id) \
            #        .filter(skc.id == str(id)) 
            #for e in q2.all():
            #    n_papers_added += 1
            #    doi = e.id.replace('doi:', '')
            #    if full_text:
            #        path = loc+db_name+'/ft/'
            #        ft_exist = download_full_text_paper_for_doi(doi, path)
            #        if ft_exist:
            #            self.db.add_full_text_for_expression(e)
            
            expression_q = self.db.session.query(func.count(ske.id)) \
                    .filter(skc_hm.has_members_id == ske.id) \
                    .filter(skc_hm.ScientificKnowledgeCollection_id == str(id)) 
            expression_count = expression_q.first()[0]

            jats_ft_q = self.db.session.query(func.count(ske.id)) \
                    .filter(skc_hm.has_members_id == ske.id) \
                    .filter(skc_hm.ScientificKnowledgeCollection_id == str(id)) \
                    .filter(ske.id == ske_hr.ScientificKnowledgeExpression_id) \
                    .filter(ske_hr.has_representation_id == ski.id) \
                    .filter(ski.type == 'JATSFullText')
            jats_ft_count = jats_ft_q.first()[0]
            
            pdf_ft_q = self.db.session.query(func.count(ske.id)) \
                    .filter(skc_hm.has_members_id == ske.id) \
                    .filter(skc_hm.ScientificKnowledgeCollection_id == str(id)) \
                    .filter(ske.id == ske_hr.ScientificKnowledgeExpression_id) \
                    .filter(ske_hr.has_representation_id == ski.id) \
                    .filter(ski.type == 'PDFFullText')
            pdf_ft_count = pdf_ft_q.first()[0]

            n = Note(id='skc:%s.counts'%(id),
                    content='{"ske_count": %s, "jats_ft_count": %s, "pdf_ft_count": %s}' \
                        %(expression_count, jats_ft_count, pdf_ft_count),
                    format='json',
                    type='NoteAboutCollection')
            c.has_notes.append(n)        
            self.db.session.add(n)

            self.db.session.commit()
            
        except Exception as e:
            raise e
        finally:  
            self.db.session.close()

        return {'report': 'We added a collection to the database called `{}` containing {} papers from this query: `{}`.'.format(name, n_papers_added, query)}

    async def _arun(
        self,
        name: str,
        query: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("add_collection_from_epmc_query does not support async")

In [ ]:
#| export

class CheckExpressionToolSchema(BaseModel):
    query: str = Field(description="a search query for the paper in the database")
    
class CheckExpressionTool(AlhazenToolMixin, BaseTool): 
    name = "describe_expression_in_local_database"
    description = "This tool searches for a paper in the database and reports if we have full text version of it."
    args_schema: Type[CheckExpressionToolSchema] = CheckExpressionToolSchema
    return_direct:bool = True

    def _run(
        self,
        query: str, 
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        
        try:      
            ske = aliased(ScientificKnowledgeExpression)
            ske_hr = aliased(ScientificKnowledgeExpressionHasRepresentation)
            ski = aliased(ScientificKnowledgeItem)

            expression_q = self.db.session.query(ske.id, ske.content, ske.type, func.aggregate_strings(ski.type, ',')) \
                    .filter(ske.id == ske_hr.ScientificKnowledgeExpression_id) \
                    .filter(ske_hr.has_representation_id == ski.id) \
                    .filter(or_(ske.id.like('%'+query+'%'), ske.content.like('%'+query+'%'))) \
                    .group_by(ske.id, ske.content, ske.type)
            df = pd.DataFrame(expression_q.all(), columns=['e_id', 'citation', 'e_type', 'i_type'])
            
        except Exception as e:
            raise e
        finally:  
            self.db.session.close()

        if len(df) == 0:
            return {'report': 'Could not find a match to query `{}`'.format(query)}
        elif len(df) > 1:
            return {'report': 'Found more than one match to query `{}`'.format(query)}
            
        p_id = None
        ft_available = False
        for i, row in df.iterrows():
            if 'FullText' in row.i_type:
                p_id = row.e_id
                ft_available = True

        return {'report': 'Found one match to query `{}` with paper_id={}, full_text_available={}'.format(query, p_id, ft_available)}
        
    async def _arun(
        self,
        name: str,
        query: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("describe_expression_in_local_database does not support async")

In [ ]:
#| export

class DescribeCollectionCompositionToolSchema(BaseModel):
    id: str = Field(description="should be the id of the collection we wish to describe")
    
class DescribeCollectionCompositionTool(AlhazenToolMixin, BaseTool): 
    name = "describe_collection_in_local_database"
    description = "This tool describes the contents of a Collection, in terms of counts of papers"
    args_schema: Type[DescribeCollectionCompositionToolSchema] = DescribeCollectionCompositionToolSchema
    return_direct:bool = True

    def _run(
        self,
        id: str, 
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        
        try:      
            collection_q = self.db.session.query(skc.name) \
                    .filter(skc.id == str(id)) 
            df = pd.DataFrame(collection_q.all(), columns=['collection_name'])
            if len(df) == 0:
                return {'report': 'The collection with the id `{}` does not exist.'.format(id)}

            name = collection_q.first()[0]

            expression_q = self.db.session.query(func.count(ske.id)) \
                    .filter(skc_hm.has_members_id == ske.id) \
                    .filter(skc_hm.ScientificKnowledgeCollection_id == str(id)) 
            expression_count = expression_q.first()[0]

            jats_ft_q = self.db.session.query(func.count(ske.id)) \
                    .filter(skc_hm.has_members_id == ske.id) \
                    .filter(skc_hm.ScientificKnowledgeCollection_id == str(id)) \
                    .filter(ske.id == ske_hr.ScientificKnowledgeExpression_id) \
                    .filter(ske_hr.has_representation_id == ski.id) \
                    .filter(ski.type == 'JATSFullText')
            jats_ft_count = jats_ft_q.first()[0]

            pdf_ft_q = self.db.session.query(func.count(ske.id)) \
                    .filter(skc_hm.has_members_id == ske.id) \
                    .filter(skc_hm.ScientificKnowledgeCollection_id == str(id)) \
                    .filter(ske.id == ske_hr.ScientificKnowledgeExpression_id) \
                    .filter(ske_hr.has_representation_id == ski.id) \
                    .filter(ski.type == 'PDFFullText')
            pdf_ft_count = pdf_ft_q.first()[0]

        except Exception as e:
            raise e
        finally:  
            self.db.session.close()

        return {'report': 'The collection with the id `{}` is called `{}`. It contains {} papers; including {} PDF and {} JATS full text papers.'.format(id, name, expression_count, pdf_ft_count, jats_ft_count)}

    async def _arun(
        self,
        name: str,
        query: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("add_collection_from_epmc_query does not support async")

In [ ]:
#| export

class DeleteCollectionToolSchema(BaseModel):
    collection_id: str = Field(description="should be the collection_id of the collection we will add papers to")

class DeleteCollectionTool(AlhazenToolMixin, BaseTool): 
    name = "delete_collection"
    description = "This deletes a collection from the database based on an collection_id value."
    args_schema: Type[DeleteCollectionToolSchema] = DeleteCollectionToolSchema
    return_direct:bool = True

    def _run(
        self,
        collection_id: str, 
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        
        try:             
            self.db.delete_collection(collection_id, commit_this=True)
        except Exception as e:
            raise e
        finally:  
            self.db.session.close()

        return {'report': 'Successfully deleted a collection with collection_id:`{}`.'.format(collection_id)}

    async def _arun(
        self,
        name: str,
        query: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("delete_collection does not support async")

In [ ]:
#| export

class RetrieveFullTextToolSchema(BaseModel):
    paper_id: str = Field(description="the digitial objecty identifier (doi) of the paper being retrieved from external sources")

class RetrieveFullTextTool(AlhazenToolMixin, BaseTool): 
    name = "retrieve_full_text_for_paper_id"
    description = "This retrieves a full text paper based on its doi, copies it to local disk, and adds it the database."
    args_schema: Type[RetrieveFullTextToolSchema] = RetrieveFullTextToolSchema
    return_direct:bool = True

    def _run(
        self,
        paper_id: str, 
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        
        loc, db_name, model_type, model_name = load_alhazen_tool_environment()

        try: 
            ske = aliased(ScientificKnowledgeExpression)
            e = self.db.session.query(ske) \
                    .filter(ske.id == str(paper_id)).first()
            doi = e.id.replace('doi:', '')
            path = loc+db_name+'/ft/'
            ft_exist = download_full_text_paper_for_doi(doi, path)
            if ft_exist:
                self.db.add_full_text_for_expression(e)
                self.db.session.commit()

        except Exception as e:
            raise e
        finally:  
            self.db.session.close()

        return {'report': 'The full text paper with doi:`{}` is available in the database.'.format(paper_id)}

    async def _arun(
        self,
        name: str,
        query: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("delete_collection does not support async")